# t-sne Visualizer

- Pull in vectors of studies
- Find $k$ reviews which have the most studies to declutter visualization
- Get rid of all studies not in those reviews
- t-sne plot

### Choose Experiment Group & ID

In [13]:
exp_group = 'siamese'
exp_id = 0

### Load Study Vectors

In [14]:
X = pickle.load(open('../store/study_vecs/{}/{}.p'.format(exp_group, exp_id))).astype('float64')

### Load Corresponding `CDNO`s

In [15]:
from trainer import Trainer

trainer = Trainer(exp_group, exp_id, hyperparam_dict={})
trainer.load_texts(inputs=['abstracts', 'outcomes'])

from sklearn.cross_validation import KFold

train_idxs, _ = next(iter(KFold(trainer.nb_train, n_folds=5, shuffle=True, random_state=1337)))
train_idxs = train_idxs[:100]

df = pd.read_csv('../../preprocess/study_inclusion.csv', index_col=0).ix[train_idxs]
cdnos = df.cdno.astype('category').cat.codes

Because a t-sne plot with this many reviews is too busy, we only keep studies which belong to the $k$ reviews with the largest number of studies.

In [16]:
K = 2

top_cdnos = set(cdnos.value_counts().sort_values(ascending=False)[:K].index)
top_cdnos

{2, 5}

Throw away studies which are not in the top $k$ reviews...

In [17]:
study_idxs = np.array(cdnos[cdnos.map(lambda cdno: cdno in top_cdnos)].index)

X = X[study_idxs]

### t-sne plot

In [ ]:
from tsne import bh_sne

X = bh_sne(X)

cmap = {i: color for i, color in enumerate(['red', 'blue'])}
plt.scatter(x=X[:, 0], y=X[:, 1], c=cdnos[study_idxs].map(cmap))